## Team CIA - Final Project Group 12
### Members: Crystal Tse, Ishita Kumar, Anna-Maria Andreeva

In [1]:
import pandas as pd
import altair as alt
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('SeekUT_TexasData_2020.xlsx', engine='openpyxl')
df.head()

inst_name   degree_level  CIP2  \
0  UT Arlington  Baccalaureate     3   
1  UT Arlington  Baccalaureate     4   
2  UT Arlington  Baccalaureate     4   
3  UT Arlington  Baccalaureate    11   
4  UT Arlington  Baccalaureate    11   

                                           CIP2Title     CIP6  \
0                 Natural Resources And Conservation   3.0104   
1                  Architecture And Related Services   4.0201   
2                  Architecture And Related Services   4.0501   
3  Computer And Information Sciences And Support ...  11.0401   
4  Computer And Information Sciences And Support ...  11.0701   

                     CIP6Title STEMStatus  \
0        Environmental Science       STEM   
1                 Architecture   NON-STEM   
2        Interior Architecture   NON-STEM   
3  Information Science/Studies       STEM   
4             Computer Science       STEM   

                                      CIP6Definition  Cohort_CNT1  \
0  A program that focuses on the application of b...         25.0   
1  A program that prepares individuals for the in...        197.0   
2  A program that prepares individuals to apply a...         55.0   
3  A program that focuses on the  theory, organiz...        531.0   
4  A program that focuses on computer theory, com...        294.0   

   NMatched_Y1  ...  DTI_aggr_Ratio_YR10  DTI_Loan_Monthly_Median_Yr1  \
0         19.0  ...                  NaN                          NaN   
1        111.0  ...                  2.3                   276.256079   
2         39.0  ...                  1.6                   250.059581   
3        384.0  ...                  NaN                   275.302708   
4        203.0  ...                  NaN                   216.302814   

   DTI_Loan_Monthly_Median_Yr5  DTI_Loan_Monthly_Median_Yr10  \
0                          NaN                           NaN   
1                   281.045871                     145.45967   
2                   202.610653                      89.68415   
3                   324.509701                           NaN   
4                   240.296427                           NaN   

   DTI_Year1wage_monthly_Median  DTI_Year5wage_monthly_Median  \
0                           NaN                           NaN   
1                   3540.341800                   4987.185894   
2                   3768.877887                   4857.173009   
3                   4395.408420                   6669.958333   
4                   5984.857055                   7543.691223   

   DTI_Year10wage_monthly_Median  firstyr_resi_tuitionfee  popular_maj_count  \
0                            NaN                      NaN               10.0   
1                    6232.302830                      NaN               54.0   
2                    5467.573651                      NaN                7.0   
3                            NaN                      NaN              189.0   
4                            NaN                      NaN              121.0   

   Loan_PCT  
0     0.556  
1     0.611  
2     0.364  
3     0.523  
4     0.407  

[5 rows x 58 columns]

In [2]:
# select only Undergraduate level
df = df[ df['degree_level'] == 'Baccalaureate']

In [3]:
# find the median wage for each year
df_yr1 = df.groupby('inst_name')['Year1Wage_Median'].median()
df_yr5 = df.groupby('inst_name')['Year5Wage_Median'].median()
df_yr10 = df.groupby('inst_name')['Year10Wage_Median'].median()

In [4]:
# append these values together so all years are combined
summary_df = pd.concat([df_yr1, df_yr5, df_yr10], axis= 1)
summary_df = summary_df.reset_index()

In [5]:
# use df_median for the 1st visualization (line graph) - get necessary columns
df_median = df[['inst_name', 'CIP6Title', 'CIP2Title',
                'Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median']]

In [6]:
# add new records for each institution for each year's median values (summary_df) to df_median
df_median = df_median.append(summary_df)
df_median

inst_name                    CIP6Title  \
0                     UT Arlington        Environmental Science   
1                     UT Arlington                 Architecture   
2                     UT Arlington        Interior Architecture   
3                     UT Arlington  Information Science/Studies   
4                     UT Arlington             Computer Science   
..                             ...                          ...   
8                 UT Permian Basin                          NaN   
9             UT Rio Grande Valley                          NaN   
10                  UT San Antonio                          NaN   
11  UT Southwestern Medical Center                          NaN   
12                        UT Tyler                          NaN   

                                            CIP2Title  Year1Wage_Median  \
0                  Natural Resources And Conservation           37991.0   
1                   Architecture And Related Services           42205.0   
2                   Architecture And Related Services           39525.0   
3   Computer And Information Sciences And Support ...           54515.0   
4   Computer And Information Sciences And Support ...           72212.0   
..                                                ...               ...   
8                                                 NaN           48747.5   
9                                                 NaN           32839.0   
10                                                NaN           37372.0   
11                                                NaN               NaN   
12                                                NaN           36521.0   

    Year5Wage_Median  Year10Wage_Median  
0                NaN                NaN  
1            60935.0            74148.0  
2            53368.0            62107.0  
3            75717.0                NaN  
4            91598.0           104794.0  
..               ...                ...  
8            55339.0            62192.5  
9            50159.0            55327.0  
10           54082.0            60897.0  
11           77977.0                NaN  
12           52448.0            62520.0  

[492 rows x 6 columns]

Note that the rows with CIP6Title as NaN are the summary ones with the median earnings for each institution.

In [7]:
# create a column called 'variable' that contains the name of the variable (wide to long conversion)
# create a column called 'value' that contains the value of said variable (salary in USD)
df_median = pd.melt(df_median, id_vars=['inst_name', 'CIP6Title', 'CIP2Title'], 
                    value_vars=['Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median'])

In [8]:
# reset the index for future index-based procedures
df_median = df_median.reset_index(drop=True)

In [9]:
# for the line graph, we need a continuous variable for the x axis
# convert the variable names to the number of years since graduation 
for i in range(0, len(df_median)):
    if df_median.loc[i, 'variable'] == 'Year1Wage_Median':
        df_median.loc[i, 'variable'] = 1
    elif df_median.loc[i, 'variable'] == 'Year5Wage_Median':
        df_median.loc[i, 'variable'] = 5
    else:
        df_median.loc[i, 'variable'] = 10
        
df_median

inst_name                    CIP6Title  \
0                       UT Arlington        Environmental Science   
1                       UT Arlington                 Architecture   
2                       UT Arlington        Interior Architecture   
3                       UT Arlington  Information Science/Studies   
4                       UT Arlington             Computer Science   
...                              ...                          ...   
1471                UT Permian Basin                          NaN   
1472            UT Rio Grande Valley                          NaN   
1473                  UT San Antonio                          NaN   
1474  UT Southwestern Medical Center                          NaN   
1475                        UT Tyler                          NaN   

                                              CIP2Title variable    value  
0                    Natural Resources And Conservation        1  37991.0  
1                     Architecture And Related Services        1  42205.0  
2                     Architecture And Related Services        1  39525.0  
3     Computer And Information Sciences And Support ...        1  54515.0  
4     Computer And Information Sciences And Support ...        1  72212.0  
...                                                 ...      ...      ...  
1471                                                NaN       10  62192.5  
1472                                                NaN       10  55327.0  
1473                                                NaN       10  60897.0  
1474                                                NaN       10      NaN  
1475                                                NaN       10  62520.0  

[1476 rows x 5 columns]

In [10]:
# rename columns for legibility for user
df_median.rename(columns = {'CIP6Title' : 'Major', 
                            'value': 'Salary', 
                            'CIP2Title' : 'Industry',
                            'variable' : 'Year'
                           }, inplace = True)

# replace NA values for 'Major' and 'Industry' with 'Institution Median' based on calculation above
df_median['Major'].fillna("Institution Median", inplace = True)
df_median['Industry'].fillna("Institution Median", inplace = True)

In [11]:
# list of unique majors
industries = df_median['Industry'].unique()

# list of institutions
institutions = df_median['inst_name'].unique()

# list of years from graduation
years = df_median['Year'].unique()

# used for study_selector (selecting industries) 
industry_df = pd.DataFrame({'Industry': industries})

# select institution dropdown
inst_dropdown = alt.binding_select(options=institutions, name = "Select Institution: ")
inst_select = alt.selection_single(
    fields=['inst_name'], 
    bind=inst_dropdown)

# checkboxes to select industry
selection = alt.selection_multi(fields=['Industry'], name = "Select Industry: ")
color = alt.condition(selection, alt.Color('Industry:N'), alt.value('lightgray'))
study_selector = alt.Chart(industry_df).mark_rect(tooltip = True).encode(
    y = alt.Y('Industry:N', title = ""), 
    color = color,
    tooltip = ['Industry']
).properties(
    title = alt.TitleParams(text = "Select Industr(ies): ", align = "right")
).add_selection(selection)

# create the salary line graph to display salaries of industries up to 10 years after graduation
salary_info = alt.Chart(df_median).mark_line(point = True, tooltip = True).encode(
    x= alt.X('Year:Q', title = "Years from Graduation", axis=alt.Axis(tickMinStep=1)),
    y= alt.Y('mean(Salary):Q', scale = alt.Scale(round = True)), 
    color= alt.Color('Industry:N', legend = None),
    tooltip = ['Industry', 'Year', 'mean(Salary)']
).properties(
    width = 500,
    title={
      "text": ["Mean Salaries After Graduation by Industry"], 
      "subtitle": ["Note: You can select the Institution Median for comparison within the checkboxes.",
                  "Press Shift + click to select multiple industries.",
                  "You can select the institution of interest at the bottom of the page."],
      "color": "black",
      "subtitleColor": "gray"
    }
).add_selection(
    inst_select
).transform_filter(
    inst_select
).transform_filter(
    selection
).interactive()

(study_selector | salary_info).save('final_viz_1of2.html')

# Visualization 2


In [12]:
# gather necessary variables for boxplot and histogram
df_industry = df[['inst_name', 'CIP6Title','CIP2Title', 'STEMStatus',
                'Year1Wage_Mean', 'Year1Wage_Median', 'Year1Wage_Min', 'Year1Wage_Max', 
                  'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75', 'Year1Wage_P90',
                  'Year5Wage_Mean', 'Year5Wage_Median', 'Year5Wage_Max' , 'Year5Wage_Min' , 
                  'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75', 'Year5Wage_P90', 
                  'Year10Wage_Mean' , 'Year10Wage_Median' , 'Year10Wage_Max' ,'Year10Wage_Min' ,
                  'Year10Wage_P10' , 'Year10Wage_P25' , 'Year10Wage_P75' , 'Year10Wage_P90']]

In [13]:
# melt dataframe from wide to long 
df_industry = pd.melt(df_industry, 
               id_vars=['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus'], 
               value_vars=['Year1Wage_Mean', 'Year1Wage_Median', 'Year1Wage_Min', 'Year1Wage_Max', 
                           'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75', 'Year1Wage_P90',
                           'Year5Wage_Mean', 'Year5Wage_Median', 'Year5Wage_Max' , 'Year5Wage_Min' , 
                           'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75', 'Year5Wage_P90', 
                           'Year10Wage_Mean' , 'Year10Wage_Median' , 'Year10Wage_Max' ,'Year10Wage_Min' ,
                           'Year10Wage_P10' , 'Year10Wage_P25' , 'Year10Wage_P75' , 'Year10Wage_P90'])

df_industry

inst_name                                CIP6Title  \
0      UT Arlington                    Environmental Science   
1      UT Arlington                             Architecture   
2      UT Arlington                    Interior Architecture   
3      UT Arlington              Information Science/Studies   
4      UT Arlington                         Computer Science   
...             ...                                      ...   
11491      UT Tyler                         Finance, General   
11492      UT Tyler              Human Resources Development   
11493      UT Tyler  Marketing/Marketing Management, General   
11494      UT Tyler                  Construction Management   
11495      UT Tyler                         History, General   

                                               CIP2Title STEMStatus  \
0                     Natural Resources And Conservation       STEM   
1                      Architecture And Related Services   NON-STEM   
2                      Architecture And Related Services   NON-STEM   
3      Computer And Information Sciences And Support ...       STEM   
4      Computer And Information Sciences And Support ...       STEM   
...                                                  ...        ...   
11491  Business, Management, Marketing, And Related S...   NON-STEM   
11492  Business, Management, Marketing, And Related S...   NON-STEM   
11493  Business, Management, Marketing, And Related S...   NON-STEM   
11494  Business, Management, Marketing, And Related S...   NON-STEM   
11495                                            History   NON-STEM   

             variable     value  
0      Year1Wage_Mean   37312.0  
1      Year1Wage_Mean   42191.0  
2      Year1Wage_Mean   42462.0  
3      Year1Wage_Mean   56053.0  
4      Year1Wage_Mean   68837.0  
...               ...       ...  
11491  Year10Wage_P90  143873.0  
11492  Year10Wage_P90   97019.0  
11493  Year10Wage_P90  148657.0  
11494  Year10Wage_P90       NaN  
11495  Year10Wage_P90   78570.0  

[11496 rows x 6 columns]

In [14]:
# create a Year column with numeric values for future use as a filter
for i in range(0, len(df_industry)):
    if df_industry.loc[i, 'variable'].startswith('Year1Wage'):
        df_industry.loc[i, 'Year'] = 1
    elif df_industry.loc[i, 'variable'].startswith('Year5Wage'):
        df_industry.loc[i, 'Year'] = 5
    else:
        df_industry.loc[i, 'Year'] = 10

In [15]:
# pivot so that year is in a separate column to enable radio button functionality
df_industry = df_industry.pivot(index = ['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus', 'Year'], 
                      columns = 'variable', 
                      values = 'value')

In [16]:
# reset index and restore columns that were used for the index for use in the visualization
df_industry.reset_index(['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus', 'Year'], inplace = True)
df_industry

variable     inst_name                                          CIP6Title  \
0         UT Arlington                                         Accounting   
1         UT Arlington                                         Accounting   
2         UT Arlington                                         Accounting   
3         UT Arlington  Aerospace, Aeronautical and Astronautical/Spac...   
4         UT Arlington  Aerospace, Aeronautical and Astronautical/Spac...   
...                ...                                                ...   
1432          UT Tyler                           Social Sciences, General   
1433          UT Tyler                           Social Sciences, General   
1434          UT Tyler                    Spanish Language and Literature   
1435          UT Tyler                    Spanish Language and Literature   
1436          UT Tyler                    Spanish Language and Literature   

variable                                          CIP2Title STEMStatus  Year  \
0         Business, Management, Marketing, And Related S...   NON-STEM   1.0   
1         Business, Management, Marketing, And Related S...   NON-STEM   5.0   
2         Business, Management, Marketing, And Related S...   NON-STEM  10.0   
3                                               Engineering       STEM   1.0   
4                                               Engineering       STEM   5.0   
...                                                     ...        ...   ...   
1432                                        Social Sciences   NON-STEM   5.0   
1433                                        Social Sciences   NON-STEM  10.0   
1434        Foreign Languages, Literatures, And Linguistics   NON-STEM   1.0   
1435        Foreign Languages, Literatures, And Linguistics   NON-STEM   5.0   
1436        Foreign Languages, Literatures, And Linguistics   NON-STEM  10.0   

variable  Year10Wage_Max  Year10Wage_Mean  Year10Wage_Median  Year10Wage_Min  \
0                    NaN              NaN                NaN             NaN   
1                    NaN              NaN                NaN             NaN   
2               415601.0          90354.0            84543.0         21624.0   
3                    NaN              NaN                NaN             NaN   
4                    NaN              NaN                NaN             NaN   
...                  ...              ...                ...             ...   
1432                 NaN              NaN                NaN             NaN   
1433                 NaN              NaN                NaN             NaN   
1434                 NaN              NaN                NaN             NaN   
1435                 NaN              NaN                NaN             NaN   
1436            355727.0          61520.0            48647.0         16443.0   

variable  Year10Wage_P10  ...  Year1Wage_P75  Year1Wage_P90  Year5Wage_Max  \
0                    NaN  ...        59194.0        67766.0            NaN   
1                    NaN  ...            NaN            NaN       219817.0   
2                49182.0  ...            NaN            NaN            NaN   
3                    NaN  ...        68960.0        76760.0            NaN   
4                    NaN  ...            NaN            NaN       145701.0   
...                  ...  ...            ...            ...            ...   
1432                 NaN  ...            NaN            NaN            NaN   
1433                 NaN  ...            NaN            NaN            NaN   
1434                 NaN  ...        39825.0        52982.0            NaN   
1435                 NaN  ...            NaN            NaN       135495.0   
1436             35084.0  ...            NaN            NaN            NaN   

variable  Year5Wage_Mean  Year5Wage_Median  Year5Wage_Min  Year5Wage_P10  \
0                    NaN               NaN            NaN            NaN   
1                70074.0           68225.0        13569.0       

In [17]:
# rename columns to eliminate Year classifier in each name - allow radio buttons to filter by Year
max_cols = ['Year1Wage_Max', 'Year5Wage_Max', 'Year10Wage_Max']
mean_cols = ['Year1Wage_Mean', 'Year5Wage_Mean', 'Year10Wage_Mean']
median_cols = ['Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median']
min_cols = ['Year1Wage_Min', 'Year5Wage_Min', 'Year10Wage_Min']

p10_cols = ['Year1Wage_P10', 'Year5Wage_P10', 'Year10Wage_P10']
p25_cols = ['Year1Wage_P25', 'Year5Wage_P25', 'Year10Wage_P25']
p75_cols = ['Year1Wage_P75', 'Year5Wage_P75', 'Year10Wage_P75']
p90_cols = ['Year1Wage_P90', 'Year5Wage_P90', 'Year10Wage_P90']


df_industry['max'] = pd.to_numeric(df_industry[max_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['mean'] = pd.to_numeric(df_industry[mean_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['median'] = pd.to_numeric(df_industry[median_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['min'] = pd.to_numeric(df_industry[min_cols].bfill(axis=1).iloc[:, 0], errors='coerce')

df_industry['p10'] = pd.to_numeric(df_industry[p10_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p25'] = pd.to_numeric(df_industry[p25_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p75'] = pd.to_numeric(df_industry[p75_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p90'] = pd.to_numeric(df_industry[p90_cols].bfill(axis=1).iloc[:, 0], errors='coerce')

In [18]:
alt.data_transformers.disable_max_rows()

# establish the color mapping
scale = alt.Scale(domain=['STEM', 'NON-STEM'],
                  range=['#5ab4ac', '#d8b365'])

# radio buttons to select year
years = df_industry['Year'].unique()
year_radio = alt.binding_radio(options=years, name="Select Years from Graduation: ")
year_select = alt.selection_single(
    fields=['Year'], 
    bind=year_radio, 
    init = {'Year': 1})

year_color_condition = alt.condition(year_select,
                      alt.Color('Year:Q', legend=None),
                      alt.value('lightgray'))

#list of institutions
institutionss = df_industry['inst_name'].unique()

# select institution dropdown
inst_dropdowns = alt.binding_select(options=institutionss, name="Select Institution: ",)
inst_selects = alt.selection_single(
    fields=['inst_name'], 
    bind=inst_dropdowns, 
    init = {'inst_name' : "UT Austin"}
)

# create the histogram to show all industries and median salaries filtered by institution and year
interval = alt.selection_multi(on='click', encodings=['y'])
chart = alt.Chart(df_industry).mark_bar().encode(
    y = alt.Y('CIP2Title:N', axis=alt.Axis(title='Industry')),
    x = alt.X('mean(median):Q', axis=alt.Axis(title='Salary')),
    color = alt.condition(interval, 'STEMStatus:N', alt.value('lightgray'), scale=scale),
    tooltip = [alt.Tooltip('CIP2Title:N', title = "Industry"),
               alt.Tooltip('mean(median):Q', title = "Median Salary"),
               alt.Tooltip('Year:Q')]
    
).properties(
    width = 600,
    title={
      "text": ["Salary Distribution by Industry"], 
      "subtitle": ["Note: Select industry(ies) on the histogram to see the distribution of associated majors as boxplots below.",
                  "Press Shift + click to select multiple industries.",
                  "You can select the institution and year from graduation at the bottom of the page."],
      "color": "black",
      "subtitleColor": "gray"
    }
    
).add_selection(
    interval,
    inst_selects,
    year_select
).transform_filter(
    inst_selects
).transform_filter(
    year_select
).interactive()


# create multiple boxplots, cannot use mark_boxplot due to aggregated values
# LayerChart generates sets of boxplots using rule, bar, and tick
# boxplots are linked to the histogram, and filtered by industry
box = alt.LayerChart(df_industry).encode(
tooltip = [alt.Tooltip('inst_name:N', title = "Institution"), 'Year',
           alt.Tooltip('CIP6Title:N', title = "Major"),
           alt.Tooltip('min:Q', title = "Minimum"), 
           alt.Tooltip('p10:Q', title = "10th Percentile"), 
           alt.Tooltip('p25:Q', title = "25th Percentile"),  
           alt.Tooltip('median:Q', title = "Median"), 
           alt.Tooltip('p75:Q', title = "75th Percentile"),  
           alt.Tooltip('p90:Q', title = "90th Percentile"),  
           alt.Tooltip('max:Q', title = "Maximum") 
          ],
y = alt.Y('CIP6Title:N', title = 'Majors')
).add_layers(
    # rule chart uses 10th and 90th percentiles instead of the standard minimum and maximum of a boxplot to exclude extreme outliers 
    alt.Chart().mark_rule(tooltip=True).encode(x=alt.X('median(p10):Q', title='Salary in Dollars ($)'), x2='min(p90):Q').interactive(),
    # bar chart creates the 25th and 75th percentile box
    alt.Chart().mark_bar(size = 15, height = 20, tooltip=True).encode(x='mean(p25):Q', x2='mean(p75):Q', color=alt.Color('STEMStatus:N')).interactive(),
    # tick chart adds the median indicator with a white line
    alt.Chart().mark_tick(color='white', height=15,tooltip=True).encode(x='mean(median):Q').interactive()
).transform_filter(
    interval
).transform_filter(
    inst_selects
).transform_filter(
    year_select
).properties(
    width = 600,
    title={
      "text": ["Distribution of Salaries of Majors for Above Selection"], 
      "subtitle": ["Note: The 10th and 90th percentiles are displayed on the boxplot to exclude extreme outliers."],
      "color": "black",
      "subtitleColor": "gray"
    }
    
).interactive()

(chart & box.add_selection(alt.selection_single())).save('final_viz_2of2.html')